# 웹 크롤링으로 코로나 사이트 크롤링 후 엑셀 파일 저장하기

In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome("C:/chromedriver_win32/chromedriver.exe")
driver.get('https://coronaboard.kr/')

보통의 경우 바로 urlopen을 한 후 beautifulsoup으로 파싱하는데 이 사이트의 경우 열어지지 않았다.
이유를 살펴보니 동적 웹사이트인 경우 크롤링이 되지 않는다는데, 동적 웹사이트와 정적 웹사이트의 큰 차이점은
사이트 내에서 계속 업데이트가 일어나는 것이라고 간단히 이해했다. 주식 정보, 뉴스 댓글 등등.. 계속 업데이트가 나는 사이트들은
동적 웹사이트에 들어간다고 보면 되겠다. 코로나보드 사이트의 경우도 이에 포함되었다.

따라서 구글링 결과 나왔던 Selenium 라이브러리를 이용하여 브라우저의 동작을 간단히 제어할 수 있는 방법을 사용하였다.

In [2]:
html=driver.page_source
soup=BeautifulSoup(html)

 그 다음 과정은 다른 크롤링 방법과 똑같았다.

In [4]:
soup.select("#country-table > div > div > table > tbody > tr:nth-child(1) > td:nth-child(2)")[0].text

'미국🇺🇸'

리스트를 이중으로 사용하여 코로나 보드 행에 해당하는 데이터들을 넣어주었다.

In [5]:
import pandas as pd
info_list=[]
for i in range(1,76):
    line_list=[] # 상응하는 순위, 국가, 확진자 수등을 각각 리스트에 넣어 사용하기 편하도록 리스트 두개 사용
    for j in range(2,8):
        text=soup.select("#country-table > div > div > table > tbody > tr:nth-child("+str(i)+")> td:nth-child("+str(j)+")")[0].text
        line_list.append(text)
    info_list.append(line_list)

출력하여 확인

In [6]:
info_list

[['미국🇺🇸',
  '9,318,043(+97,615)',
  '235,180(+848)',
  '6,025,295(+40,078)',
  '2.5',
  '64.7'],
 ['인도🇮🇳',
  '8,137,119(+47,391)',
  '121,681(+537)',
  '7,432,829(+59,454)',
  '1.5',
  '91.3'],
 ['브라질🇧🇷',
  '5,519,528(+23,126)',
  '159,562(+529)',
  '4,966,264(+12,105)',
  '2.9',
  '90.0'],
 ['러시아🇷🇺',
  '1,618,116(+18,140)',
  '27,990(+334)',
  '1,215,414(+14,854)',
  '1.7',
  '75.1'],
 ['프랑스🇫🇷',
  '1,331,984(+49,215)',
  '36,565(+545)',
  '116,533(+1,246)',
  '2.7',
  '8.7'],
 ['스페인🇪🇸', '1,264,517(+25,595)', '35,878(+239)', 'N/A', '2.8', 'N/A'],
 ['아르헨티나🇦🇷',
  '1,157,179(+13,379)',
  '30,792(+350)',
  '961,101(+14,967)',
  '2.7',
  '83.1'],
 ['콜롬비아🇨🇴',
  '1,063,151(+10,029)',
  '31,135(+209)',
  '959,433(+9,085)',
  '2.9',
  '90.2'],
 ['영국🇬🇧', '989,745(+24,405)', '46,628', 'N/A', '4.7', 'N/A'],
 ['멕시코🇲🇽',
  '918,811(+6,000)',
  '91,289(+516)',
  '673,457(+4,790)',
  '9.9',
  '73.3'],
 ['페루🇵🇪', '900,180(+2,586)', '34,411(+49)', '822,316(+2,599)', '3.8', '91.4'],
 ['남아프리카 공화국🇿🇦',
  '723

추출한 데이터들을 데이터프레임으로 표 형식으로 바꿔 준 뒤 columns에 네이밍을 해주었다.

In [7]:
datalist=pd.DataFrame(info_list,columns=['순위','국가','확진자','사망자','사망률','완치'])

다시 출력하여 확인

In [8]:
datalist

,순위,국가,확진자,사망자,사망률,완치
0,미국🇺🇸,"9,318,043(+97,615)","235,180(+848)","6,025,295(+40,078)",2.5,64.7
1,인도🇮🇳,"8,137,119(+47,391)","121,681(+537)","7,432,829(+59,454)",1.5,91.3
2,브라질🇧🇷,"5,519,528(+23,126)","159,562(+529)","4,966,264(+12,105)",2.9,90.0
3,러시아🇷🇺,"1,618,116(+18,140)","27,990(+334)","1,215,414(+14,854)",1.7,75.1
4,프랑스🇫🇷,"1,331,984(+49,215)","36,565(+545)","116,533(+1,246)",2.7,8.7
...,...,...,...,...,...,...
70,알제리🇩🇿,"57,651(+319)","1,956(+7)","40,014(+379)",3.4,69.4
71,아제르바이잔🇦🇿,"54,174",718,"42,996",1.3,79.4
72,케냐🇰🇪,"53,797(+1,185)",981(+17),"35,876(+272)",1.8,66.7
73,팔레스타인🇵🇸,"53,075",481,"45,896",0.9,86.5


마무리로 한글이 깨지지 않도록 'utf-8-sig' 인코딩 해준 후 .to_csv로 엑셀파일로 저장해주었다. 

In [10]:
datalist.to_csv("국가별 현황.csv",index=False,encoding='utf-8-sig')